In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Research3/train

/content/drive/MyDrive/Research3/train


In [2]:
import torch
from torch import nn
from torch.nn.parameter import Parameter

In [3]:
import numpy as np
import pandas as pd
import random
from torch import multiprocessing

In [7]:
def make_lag(n_X, n_y, y):
    n = y.shape[0]+1-n_X-n_y
    if n <= 0: raise ValueError
    X = torch.empty((n, n_X))
    Y = torch.empty((n, n_y))
    for i in range(n_X):
        X[:, i] = y[i:n+i]
    for i in range(n_y):
        Y[:, i] = y[n_X+i:n+n_X+i]
    return X, Y

In [8]:
df = pd.read_csv('../rawdata/sunspots.csv', index_col='Unnamed: 0')
df.columns = ['Date', 'Sunspot']
y = df['Sunspot'].to_numpy()
y.shape

(3252,)

In [9]:
X, Y = make_lag(11*12, 10, torch.FloatTensor(y))
X = X.unsqueeze(-1).transpose(0, 1)
X.shape, Y.shape

(torch.Size([132, 3111, 1]), torch.Size([3111, 10]))

In [19]:
class Layer(torch.nn.Module):
    def __init__(self, n_seq, n_input, n_hidden):
        super().__init__()
        self.n_seq = n_seq
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.rnncell = torch.nn.GRUCell(n_input, n_hidden)
        self.parent = nn.parameter.Parameter(-torch.ones(n_seq, dtype=torch.long), requires_grad=False)
        #for i in range(n_seq-1): self.parent[i] = i+1
        #self.parent = [-1 if (i == n_seq-1) else n_seq-1 for i in range(n_seq)]
        #self.parent = [-1 if (i == n_seq-1) else random.randrange(i+1, n_seq) for i in range(n_seq)]
        #self.parent = [-1 if (i == n_seq-1) else i+1 for i in range(n_seq)]
        self.childs = self._construct_childs()
        self.score = torch.zeros((n_seq, n_seq))
        
    def forward(self, X):
        if (X.dim() != 3) or (X.size(0) != self.n_seq) or (X.size(2) != self.n_input): raise ValueError()
        hx = torch.empty((self.n_seq, X.size(1), self.n_hidden), dtype=X.dtype, device=X.device)
        nchilds = [len(child) for child in self.childs]
        
        q = []
        for idx, val in enumerate(nchilds):
            if val == 0: q.append(idx)
        while len(q) > 0:
            agg = torch.cat([self._aggregate(hx[self.childs[i]]) for i in q], 0)
            print(X[q].shape)
            hx[q] = self.rnncell(X[q].reshape(-1, self.n_input), agg).reshape(len(q), -1, self.n_hidden)
            p = []
            for idx in q:
                nchilds[self.parent[idx]] -= 1
                if nchilds[self.parent[idx]] == 0:
                    p.append(self.parent[idx])
            q = p
            
        for i in range(self.n_seq): hx[i] = self.rnncell(X[i], self._aggregate(hx[self.childs[i]]))
        self.score = self.score.to(X)
        with torch.no_grad(): self.score += self._dot_product(hx)
        return hx, hx[-1]

    def add_edge(self):
        old, new = self._get_index()
        self.parent[new] = old
        self.childs = self._construct_childs()
        self.score = torch.zeros((self.n_seq, self.n_seq))
        
    def _construct_childs(self):
        childs = [[] for i in range(self.n_seq)]
        for idx in range(self.n_seq):
            if self.parent[idx] >= 0:
                childs[self.parent[idx]].append(idx)
        return childs

    def _connected_end(self):
        connected = (self.parent != -1)
        connected[-1] = True
        return connected
    
    def _aggregate(self, h):
        if h.size(0) == 0: return torch.zeros(h.shape[1:], dtype=h.dtype, device=h.device)
        else: return h.mean(axis=0)
    
    def _dot_product(self, hx):
        hx_norm = torch.div(hx, LA.norm(hx, dim=2).reshape(hx.shape[0], hx.shape[1], -1))
        return torch.tensordot(hx_norm, hx_norm, dims=([1, 2], [1, 2]))
    
    def _get_index(self):
        old = self._connected_end()
        mask = torch.ones_like(self.score, dtype=torch.bool).tril(diagonal=-1)
        mask[old != True, :] = False
        mask[:, old] = False
        maskedscore = self.score[mask] / torch.sqrt(self.score[0][0])  
        cumask = torch.cumsum(mask, dim=1)
        cucumask = torch.cumsum(cumask[:, -1], dim=0)

        maskedscore = torch.nn.functional.softmax(maskedscore, dim=0)
        #plt.plot(maskedscore.cpu())
        #plt.show()
        maskedscore = torch.cumsum(maskedscore, dim=0)
        selectcursor = torch.rand(1).to(maskedscore)
        selectedidx = (maskedscore < selectcursor).sum()
        #selectedidx = maskedscore.argmax()

        row = (cucumask <= selectedidx).sum()
        if row > 0: selectedidx -= cucumask[row-1]
        col = (cumask[row] <= selectedidx).sum()
        print(selectedidx, row, col)
        return row, col

In [20]:
l = Layer(11*12, 1, 20)

In [21]:
for i in range(10):
    l(X)
    l.add_edge()

torch.Size([132, 3111, 1])


NameError: name 'LA' is not defined

In [50]:
l(X)

tensor([[[ 2.2506e-02,  9.8297e-01, -1.1921e-07,  ...,  9.9446e-01,
           1.1086e-05, -2.7418e-06],
         [ 1.7213e-02,  9.8760e-01,  0.0000e+00,  ...,  9.9632e-01,
           4.5300e-06, -1.0729e-06],
         [ 1.1061e-02,  9.9262e-01,  0.0000e+00,  ...,  9.9812e-01,
           1.0729e-06, -2.3842e-07],
         ...,
         [ 5.3887e-02,  3.7908e-01, -1.5499e-01,  ...,  2.8277e-01,
           2.6258e-01, -1.7998e-01],
         [ 4.2095e-02,  3.0794e-01, -1.4621e-01,  ...,  1.9455e-01,
           2.3492e-01, -1.4482e-01],
         [ 1.2832e-01,  6.4648e-01, -5.8011e-02,  ...,  6.7711e-01,
           1.4647e-01, -8.7178e-02]],

        [[ 1.7213e-02,  9.8760e-01,  0.0000e+00,  ...,  9.9632e-01,
           4.5300e-06, -1.0729e-06],
         [ 1.1061e-02,  9.9262e-01,  0.0000e+00,  ...,  9.9812e-01,
           1.0729e-06, -2.3842e-07],
         [ 2.5794e-02,  9.7998e-01, -2.3842e-07,  ...,  9.9317e-01,
           1.7643e-05, -4.5300e-06],
         ...,
         [ 4.2095e-02,  3

(tensor([[[ 2.2506e-02,  9.8297e-01, -1.1921e-07,  ...,  9.9446e-01,
            1.1086e-05, -2.7418e-06],
          [ 1.7213e-02,  9.8760e-01,  0.0000e+00,  ...,  9.9632e-01,
            4.5300e-06, -1.0729e-06],
          [ 1.1061e-02,  9.9262e-01,  0.0000e+00,  ...,  9.9812e-01,
            1.0729e-06, -2.3842e-07],
          ...,
          [ 5.3887e-02,  3.7908e-01, -1.5499e-01,  ...,  2.8277e-01,
            2.6258e-01, -1.7998e-01],
          [ 4.2095e-02,  3.0794e-01, -1.4621e-01,  ...,  1.9455e-01,
            2.3492e-01, -1.4482e-01],
          [ 1.2832e-01,  6.4648e-01, -5.8011e-02,  ...,  6.7711e-01,
            1.4647e-01, -8.7178e-02]],
 
         [[ 1.7213e-02,  9.8760e-01,  0.0000e+00,  ...,  9.9632e-01,
            4.5300e-06, -1.0729e-06],
          [ 1.1061e-02,  9.9262e-01,  0.0000e+00,  ...,  9.9812e-01,
            1.0729e-06, -2.3842e-07],
          [ 2.5794e-02,  9.7998e-01, -2.3842e-07,  ...,  9.9317e-01,
            1.7643e-05, -4.5300e-06],
          ...,
    

In [48]:
l.parent

Parameter containing:
tensor([131, 131, 131, 131, 131, 131, 131, 131, 131, 131,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1])

In [ ]:
g = torch.nn.GRU(1, 20, batch_first=True)

In [ ]:
%timeit l(X)

34.6 ms ± 529 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit g(X)

19.4 ms ± 324 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
